# Demo notebook: Clojupyter in Jupyter Lab

This notebook demonstrates some features of Clojupyter for Jupyter Lab.  Please note that this notebook is intended to be used with **Jupyter Lab**.


In [ ]:
(require '[clojupyter.display :as display])
(require '[cheshire.core :as json])
(require '[clojure.data.codec.base64 :as b64])
:ok

Helper function to read bytes from uri string

In [ ]:
(defn slurp-bytes
  "Slurp the bytes from a slurpable thing"
  [x]
  (with-open [out (java.io.ByteArrayOutputStream.)]
    (clojure.java.io/copy (clojure.java.io/input-stream x) out)
    (.toByteArray out)))

## Vega Lite
For this demo we define some simple functions generating some data compatible with the high-level  charting library:

In [ ]:
(defn datapt [[s n]] {:Index s, :Value n})
(defn graph  [vs]    {:data {:values vs}, 
                      :mark :line
                      :width 600
                      :height 400
                      :encoding {:x {:field :Index :type :ordinal}
                                 :y {:field :Value :type :quantitative}}
                      :$schema "https://vega.github.io/schema/vega-lite/v3.json"})
(def graph-it (comp graph (partial map datapt) (partial apply map vector)));

With a generator for (slightly) random data...

In [ ]:
(defn raw-data [] (map #(+ % (rand-int 25)) (range)))

...we can create a graph using Vega Lite:

In [ ]:
(->> [(range) (take 180 (raw-data))] graph-it display/vega-lite)

**Note**: The above cell is *not* supposed to render correctly in Jupyter Notebook, <font color=red>**only Jupyter Lab**</font>.

## Vega
The Vega spec is overly verbose and generating one by hand is outside the scope of this demo. Clojupyter can render Clojure maps that serialize to Vega JSON spec using `clojupyter.display/vega`.

In [ ]:
(-> "https://raw.githubusercontent.com/vega/vega/master/docs/examples/bar-chart.vg.json" slurp json/parse-string display/vega-lite)

**Jupyter Lab** supports other mime types by default:
* `:text/markdown` `:text/latex`
* `:image/bmp` `:image/gif` `:image/jpeg` `:image/png` `:image/svg+xml`
* `:application/json` `:application/pdf` `:application/vdom.v1+json`

## JSON
Clojure maps that can be serialized to JSON can be viewed as `:application/json`

In [ ]:
(display/json {:name "JSON Map" :fields [{:name "Field0" :position 0} {:name "Field1" :position 1}]})

## VDom
VDom, short for Virtual Dom is a way to generate html content from JSON maps.

In [ ]:
(display/vdom {:tagName :p :attributes {:style {:color :red}} :children "Hello World!"})